In [36]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import tensorflow as tf
tf.__version__

'2.2.0-rc3'

In [38]:
import pandas as pd

In [39]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [40]:
from bert.tokenization.bert_tokenization import FullTokenizer

In [41]:
import sys
import os
import re
import pandas as pd
import pickle
sys.path.append('/Users/keemsunguk/Projects/EssayGrader/')

In [72]:
from egrader.data_util import load_dataset, EssayGraderData
from egrader.bert_util import create_model
from egrader.train import train_essay_grader
from egrader.predict import predict_essay_grade 

In [43]:
from egrader.data_util import load_essay_from_db

In [44]:
from egrader.db_util import DBUtil

In [45]:
bert_ckpt_dir    = '/Users/keemsunguk/Projects/data/uncased_L-12_H-768_A-12'
bert_ckpt_file   = '/Users/keemsunguk/Projects/data/uncased_L-12_H-768_A-12/bert_model.ckpt'
bert_config_file = '/Users/keemsunguk/Projects/data/uncased_L-12_H-768_A-12/bert_config.json'

In [57]:
%%time

tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))
#data = EssayGraderData(tokenizer, 
#                       sample_size=10*768*12,
#                       max_seq_len=512)
with open('/Users/keemsunguk/Projects/data/toefl_balanced.pkl', 'rb') as rf:
    data_df = pickle.load(rf)

CPU times: user 148 ms, sys: 133 ms, total: 281 ms
Wall time: 354 ms


In [58]:
data = EssayGraderData(tokenizer, essay_df=data_df, sample_size=10*768*12, max_seq_len=512)

Total data: 14037



max seq_len 1320


In [59]:
with open('/Users/keemsunguk/Projects/data/toefl_data_balanced.pkl', 'wb') as wf:
    pickle.dump(data, wf)

In [60]:
with open('/Users/keemsunguk/Projects/data/toefl_data_balanced.pkl', 'rb') as rf:
    data = pickle.load(rf)

In [61]:
print("            train_x", data.train_x.shape)
print("train_x_token_types", data.train_x_token_types.shape)
print("            train_y", data.train_y.shape)
print("             test_x", data.test_x.shape)
print("        max_seq_len", data.max_seq_len)

            train_x (9825, 512)
train_x_token_types (9825, 512)
            train_y (9825,)
             test_x (4212, 512)
        max_seq_len 512


In [68]:
adapter_size = None # use None to fine-tune all of BERT
model = create_model(data.max_seq_len, bert_config_file, bert_ckpt_file, adapter_size=adapter_size)

bert shape (None, 512, 768)
Done loading 196 BERT weights from: /Users/keemsunguk/Projects/data/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x169560e50> (prefix:bert_3). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 512)]             0         
_________________________________________________________________
bert (BertModelLayer)      

In [69]:
model.save('/Users/keemsunguk/Projects/data/init_essay_model.h5', overwrite=True)

In [70]:
# From here, training need GPU

In [ ]:
model = train_essay_grader(model, data, checkpoint_path= '/Users/keemsunguk/Projects/data', batch_size=8, validation_split=0.5, epoch=10)


Epoch 00001: LearningRateScheduler reducing learning rate to 5.000000000000001e-07.
Epoch 1/10
186/614 [========>.....................] - ETA: 6:03:26 - loss: 1.7733 - acc: 0.2191

In [ ]:
model.save_weights('/Users/keemsunguk/Projects/data/essay_weights_10_768_12_epoc10.h5', overwrite=True)

In [ ]:
model.save('/Users/keemsunguk/Projects/data/essay_model_10_768_12_epoc10.h5', overwrite=True)

In [33]:
%%time

_, train_acc = model.evaluate(data.train_x, data.train_y)
_, test_acc = model.evaluate(data.test_x, data.test_y)

print("train acc", train_acc)
print(" test acc", test_acc)

44/44 [==============================] - 5s 108ms/step - loss: 1.5841 - acc: 0.5842
train acc 0.594921886920929
 test acc 0.5842371582984924
CPU times: user 44.5 s, sys: 20.1 s, total: 1min 4s
Wall time: 14.2 s


In [37]:
pred_sentences = [
  "0112 (pr) some people think that children should begin their formal education at a very early age and should spend most of their time on school studies. others believe that young children should spend most of their time playing. compare these two views. which view do you agree with? why?. there is a trend of early education in korea. most of the parents start teaching their children things like math, and english, as early as children are 3 years old. the parents think early education influence greatly on chilldren's future. however, there are a few other parents that are against early education. they think children should play in their childhood, not study. let me explain you advantages of each one.    the children with early education, in fact, do great work in school after they grow up. they have more knowledge than average students, and also shows better ability in math, and english. moreover, it's always the early educated children who gets the first place in their class.    on the other hand, children that spent most of the time playing, have relatively low score at schools, and they have lower chance of going to good, well-known schools. however, there is something special to the children that played in early ages. they have more creativity, and knowledge about social relationship among people. while playing games with other kids, the children find out how the relationship should be made, and they grow their imagination.    in conclusion, i think children should play in their childhood, although they may get bad scores on school work. they can broaden their imagination, and relationship, which will influence them greatly in the future. thus, young children need playing to grow their potential.",
#  "The acting was a bit lacking",
#  "The film was creative and surprising",
#  "Absolutely fantastic!"
]

res = predict_essay_grade(model, data, bert_ckpt_dir, pred_sentences)
for text, rating in zip(pred_sentences, res):
    print(" text:", text)
    print("  res:", rating)

pred_token_ids (1, 342)


ValueError: in user code:

    /Users/keemsunguk/base37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1150 predict_function  *
        outputs = self.distribute_strategy.run(
    /Users/keemsunguk/base37/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/keemsunguk/base37/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/keemsunguk/base37/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/keemsunguk/base37/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1125 predict_step  **
        return self(x, training=False)
    /Users/keemsunguk/base37/lib/python3.7/site-packages/tensorflow/python/keras/e